# Personal Finance Subreddit Capstone Project

## Data Wrangling

The goal of this notebook is to extract data containing information about the personal finance subreddit from the pushshift.io Reddit API, select interesting features that can be relevant to the project and clean the dataset so that it is ready for data exploration.

- **Import the necessary packages**

In [ ]:
import praw
import pandas as pd
import numpy as np
import datetime
import json
import requests
import string
import time
import datetime
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

### Data Acquisition

- **Extract the data from reddit using pushshift.io's API**

In [ ]:
# Variables
sub    = 'personalfinance'     # name of the subreddit you would like to scrape
after  = '2018-08-10'    # earliest date that will be scraped
before = '2018-08-25'    # latest date that will be scraped
fast   = True           # True will be faster, won't pull upvote ratio

In [ ]:
# Initiate sqlite
sql = sqlite3.connect('personalfinance_.db')
cur = sql.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS posts (name TEXT, title TEXT, readable_utc TEXT, permalink TEXT, domain TEXT, url TEXT, author TEXT, score TEXT, upvote_ratio TEXT, num_comments TEXT)')
sql.commit()
print('Loaded SQL Database and Tables')

# Convert the specified dates to strptime
after = time.mktime(datetime.datetime.strptime(after, '%Y-%m-%d').timetuple())
after = int(after)
readable_after = time.strftime('%d %b %Y %I:%M %p', time.localtime(after))
before = time.mktime(datetime.datetime.strptime(before, '%Y-%m-%d').timetuple())
before = int(before) + 86399
readable_before = time.strftime('%d %b %Y %I:%M %p', time.localtime(before))
print('Searching for posts between ' + readable_after + ' and ' + readable_before + '.')
currentDate = before

Using pushshift will allow us to retrieve valuable information from reddit submissions including:
- Submission ID
- Submission title
- Submission date
- Submission permalink
- Submission domain
- Submission url
- Submission author
- Submission score (upvotes)
- Submission upvote_ratio (ratio of upvotes to downvotes)
- Submission number of comments

However, it does not provide us with the flair information.

NOTE: This process is extremely computationally expensive.

In [ ]:
# Perform a new full pull from Pushshift
def newpull(thisBefore):
    global currentDate
    readable_thisBefore = time.strftime('%d %b %Y %I:%M %p', time.localtime(thisBefore))
    print('Searching posts before ' + str(readable_thisBefore))
    url = 'http://api.pushshift.io/reddit/search/submission/?subreddit=' + sub + '&size=500&before=' + str(thisBefore)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print('    Discussion: HTML Error - ', response.status_code)
        time.sleep(60)
        return
    curJSON = response.json()

    # Update each Pushshift result with Reddit data
    for child in curJSON['data']:

        # Check to see if already added
        name = str(child['id'])
        cur.execute('SELECT * FROM posts WHERE name == ?', [name])
        if cur.fetchone():
            print(str(child['id']) + ' skipped (already in database)')
            continue

        # If not, get more data
        if fast is True:
            searchURL = 'http://reddit.com/by_id/t3_'
        else:
            searchURL = 'http://reddit.com/'
        url = searchURL + str(name) + '.json'
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print('    Discussion: HTML Error - ', response.status_code)
            time.sleep(60)
            break
        postJSON = response.json()
        if fast is True:
            jsonStart = postJSON
        else:
            jsonStart = postJSON[0]

        # Check to see if Date has passed
        global currentDate
        created_utc = jsonStart['data']['children'][0]['data']['created_utc']
        currentDate = int(created_utc)
        if currentDate <= after:
            break

        # If not, process remaining data
        try:
            title = str(jsonStart['data']['children'][0]['data']['title'])  # Checks for emojis and other non-printable characters
        except UnicodeEncodeError:
            title = ''.join(c for c in str(jsonStart['data']['children'][0]['data']['title']) if c in string.printable)
        readable_utc = time.strftime('%d %b %Y %I:%M %p', time.localtime(created_utc))
        permalink    = (str(jsonStart['data']['children'][0]['data']['permalink']))
        domain       = (str(jsonStart['data']['children'][0]['data']['domain']))
        url          = (str(jsonStart['data']['children'][0]['data']['url']))
        author       = (str(jsonStart['data']['children'][0]['data']['author']))
        score        = (str(jsonStart['data']['children'][0]['data']['score']))
        num_comments = (str(jsonStart['data']['children'][0]['data']['num_comments']))
        if fast is True:
            upvote_ratio = 0
        else:
            upvote_ratio = (str(jsonStart['data']['children'][0]['data']['upvote_ratio']))

        # Write it to SQL Database
        cur.execute('INSERT INTO posts VALUES(?,?,?,?,?,?,?,?,?,?)', [name, title, readable_utc, permalink, domain, url, author, score, upvote_ratio, num_comments])
        sql.commit()

# Run the newpull
while currentDate >= after:
    newpull(currentDate)

Now that we have pulled the data from pushshift, we will need to create a dataframe which will store the relevant information (title, date, time, upvotes, id).

In [ ]:
# Reconnect to sqlite
connection = sqlite3.connect("personalfinance_.db") 
  
# Cursor object 
crsr = connection.cursor() 
  
# Execute the command to fetch all the data from the table posts 
crsr.execute("SELECT * FROM posts")  
  
# Store all the fetched data in the ans variable 
ans= crsr.fetchall()  

# Create empty dataframe
columns = ['title']
index = range(0,2)
df = pd.DataFrame(index = index, columns = columns)
df = df.fillna(0)

# Create new columns and extract the relevant data 
for n, i in enumerate(ans):
    # Create title column
    df.loc[n , 'title'] = i[1]
    # Create date column
    df.loc[n , 'date'] = i[2][:-8]
    # Create time column
    df.loc[n , 'time'] = i[2][-8:]
    # Create upvote column
    df.loc[n , 'upvotes'] = i[7]
    # Create id column
    df.loc[n , 'id'] = i[0]

- **Retrieve flair information from Reddit's API**

As mentioned before, we still need to extract the flair (which indicates the topic of each submission) from each post. To do this, we will have to initiate a Reddit instance using praw (which gives access to Reddit's API).

In [ ]:
# Create new Reddit instance
reddit = praw.Reddit(client_id='',
                     client_secret='',
                     user_agent='',
                    username = '',
                    password = '')

Create a for-loop that will check and return the appropriate flair for each submission by using its ID as verification.

In [ ]:
for a,b in enumerate(df.id):
    df.loc[a, 'topic'] = reddit.submission(id = "{}".format(b)).link_flair_text
    try:
        print(a,',', df.loc[a, 'topic'])
    except:
        print('None')

- **Retrieve self-text information**

Most posts contain text where the user explains what their issue(s) is. Although some posts may not contain any text due to it being removed or deleted, it can still give better insight than to use only the titles.

In [ ]:
for a,b in enumerate(df.id):
    df.loc[a, 'self_text'] = reddit.submission(id = "{}".format(b)).selftext
    try:
        print(a,',', df.loc[a, 'self_text'])
    except:
        print('None')

Let's replace the self-text entries where there are NaN values.

In [ ]:
for n, x in enumerate(df['self_text']):
    if pd.isnull(x):
        df.loc[n, 'self_text'] = ''

Some of the posts have been removed or deleted at the time that we extracted the data so they contain the strings '[removed]' and '[deleted]' inside the self-text. We will need to remove these.

In [ ]:
for n, x in enumerate(df['self_text']):
    if x in ['[removed]', '[deleted]']:
        df.loc[n, 'self_text'] = ''

- **Concatenate the submission title and self-text**

By combining the submission title and self-text for each post, we will have a better idea of what the topics are.

In [ ]:
df['text'] = df['title'] + str(' ') + df['self_text']

- **Convert Dates into datetime format**

Since the data is given as a string, we will need to convert it into datatime format.

In [ ]:
from datetime import datetime
for a,b in enumerate(df.date):
     df.loc[a, 'date'] = datetime.strptime(b, '%d %b %Y ').date()

In [ ]:
df = df.sort_values(by=['date', 'time'], ascending = False).reset_index().drop('index',axis='columns')

- **Examine basic information**

Let's begin by taking a peek at the dataframe's contents.

In [23]:
print('Dimensions of the dataframe: {}'.format(df.shape))
print(20*'-')
df.info()

Dimensions of the dataframe: (13001, 8)
--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13001 entries, 0 to 13000
Data columns (total 8 columns):
title        13001 non-null object
date         13001 non-null object
time         13001 non-null object
upvotes      13001 non-null object
id           13001 non-null object
topic        12258 non-null object
self_text    13001 non-null object
text         13001 non-null object
dtypes: object(8)
memory usage: 812.6+ KB


The only column with missing data is 'topic' due to the fact that some posts have been removed and therefore their flairs no longer show up during extraction. To deal with this issue, we can simply fill in 'unknown'.

In [24]:
df['topic'].fillna('unknown', inplace = True)

In [25]:
df.topic.value_counts()

Debt          1563
Other         1544
Credit        1399
Investing     1051
Retirement     990
Employment     964
Housing        865
Auto           743
unknown        743
Planning       703
Saving         676
Taxes          655
Budgeting      608
Insurance      497
Name: topic, dtype: int64

Replace the outlier topics with the topic 'unknown'.

In [6]:
outliers = df[(df['topic'] == 'Meta' )| (df['topic'] == 'THIS IS A SPAMMER')]['topic']

for id_ in outliers.index:
    df.loc[id_,'topic'] = 'unknown'

### Data Pre-processing

Pre-processing is an important part of machine learning and even more significant for Natural Language Processing tasks because a simple error could result in catastrophic mishaps. 

There are 3 major components in data pre-processing:

**1) Data-cleaning**: We will need to first clean up the text through various steps:

- **Lowercase** the words so that the model will not differentiate capitalized words from other words.

- **Remove numbers/digits** since the model is interpreting *text* not numbers.

- **Remove punctuation** since it is not important for the context.

- **Strip white space** since empty strings could be interpreted as text and we want to avoid that.

- **Remove stopwords**, which are general words that are very frequent in the English dictionary (ex. because, such, so). Here is a list of some common stopwords: https://www.ranks.nl/stopwords

- **Remove noise** that is not picked up through the other cleaning methods. This step can come either before or after tokenization and normalization, or both (ex. dropping words that are less than 2 characters long).

**2) Tokenization**: In order to better analyze individual words, we will need to *tokenize* the documents (or in this case, the submission titles) into pieces of words. By doing so, we will be able to use the various NLP libraries to further dissect the tokens.

**3) Normalization**: After tokenizing the data, we will need to normalize the text through lemmatization and stemming. Lemmatization is typically a better method since it returns the canonical forms based on a word's lemma. However, this process takes much more time compared to stemming the words, which simply removes the affixes of a given word.

In [1]:
import string
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

def preprocess(text):
    # Replace the forward slash with space
    text = text.replace('/', ' ')
    # Remove all other punctuation without replacement
    text = ''.join([char for char in text if char not in string.punctuation])
    # Remove digits (excluding strings that contain both digits and letters)
    text = ''.join([char for char in text if char not in string.digits])
    # Strip whitespaces
    text = ' '.join(text.split())
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stopWords])
    # Lowercase all words
    text = text.lower()
    # Return only words that have more than 2 letters
    text = ' '.join([word for word in text.split() if len(word)>2])
    # Remove any symbols or non-alphabetical letters
    text = re.sub('[^ a-zA-Z]', '', text)
    
    return text

In [6]:
df['clean_text'] = df['text'].apply(lambda x: preprocess(x))
df['clean_text'].head(10)

0                           ways make extra side money
1    year update legally blind going homeless one j...
2    kicked found last night home ive staying going...
3    online savings account hello looking recommend...
4                      tools managing incomes expenses
5    with resources like reddit financial consultin...
6    credit hit late payment fee waiver score refle...
7    need help budgetting getting debt long story s...
8    year old male almost two year fiance recently ...
9        debt collector gave hours pay yelled said ssn
Name: clean_text, dtype: object

In [7]:
# Check to see all clean text are not empty
for num, x in enumerate(df.clean_text):
    if not x:
        print('Row number {} has an empty entry'.format(num))

Row number 2845 has an empty entry
Row number 3646 has an empty entry
Row number 4592 has an empty entry
Row number 4705 has an empty entry


In [35]:
# See what the issue is and correct it
df.loc[[2167, 9597]]

title        date      time  \
2167  ????????? 1 ??? ? ?????? ??? ???????? ? 2018  2018-09-14  11:33 AM   
9597                                      50/20/30  2018-08-27  11:27 PM   

      upvotes      id      topic clean_title  
2167        1  9fszwz        NaN              
9597        1  9aviyl  Budgeting

In [36]:
# Since the titles do not contain much information, let's drop them
df.drop([2167, 9597], inplace = True)

# Reset the index 
df = df.reset_index().drop('index', axis=1)

Let's check once more if there are any null or missing values in the 'clean_text' column.

In [62]:
for n, x in enumerate(df['clean_text']):
    if (pd.isnull(x)) or (not x):
        print(n, x)

2845 nan
3646 nan
4592 nan
4705 nan


In [67]:
null_list = [2845, 3646, 4592, 4705]
for x in null_list:
    print('title: {}, self_text: {}, text: {}, clean_text: {}'.format(df['title'][x], df['self_text'][x], df['text'][x],
                                                                     df['clean_text'][x]))
df.drop(null_list, inplace = True)

title: 1099 or W2?, self_text: , text: 1099 or W2? , clean_text: nan
title: 401Ks?, self_text: , text: 401Ks? , clean_text: nan
title: 401k, self_text: , text: 401k , clean_text: nan
title: T, self_text: , text: T , clean_text: nan


In [78]:
df = df.sort_values(by=['date', 'time'], ascending = False).reset_index().drop('index',axis='columns')

- **Character/text Limit**

Let's also limit the number of characters in the text since the longer the text, the longer the computation time. Rather than choosing an arbitrary cutoff point, we should explore various statistics regarding the text lengths.

In [8]:
# Compute the mean and median text lengths
txt_len_mean = df['clean_text'].map(len).mean()
txt_len_median = df['clean_text'].map(len).median()

print('Mean Text Length: {}\n\nMedian Text length: {}'.format(txt_len_mean, txt_len_median))

Mean Text Length: 435.36116295059423

Median Text length: 309.0


In [9]:
# Observe the top 10 texts with most number of characters
txt_len_list= {}
for n, x in enumerate(df['clean_text']):
    txt_len_list[n] = len(x)
sorted(txt_len_list.items(), key = lambda x: x[1], reverse = True)[0:10]

[(858, 9254),
 (1191, 7052),
 (2435, 6278),
 (6524, 6021),
 (4495, 5515),
 (2608, 5315),
 (9846, 5172),
 (9512, 4932),
 (6921, 4926),
 (5355, 4861)]

In [10]:
# Top 15 most frequent text length amounts
df['clean_text'].map(len).value_counts()[0:15]

35    76
28    75
21    74
29    74
30    71
27    70
34    68
37    68
23    67
24    66
39    65
31    65
26    64
36    61
25    60
Name: clean_text, dtype: int64

It looks like 450 characters would be a good cut-off point. What happens when we try it out on a few samples?

In [11]:
[text[0:450] for text in df['clean_text'][5:10]]

['with resources like reddit financial consulting advisement dying business seems like advantage knowing advisor breathing computer making automated decisions retirement also helps he’s professional field foreseeable future occupation',
 'credit hit late payment fee waiver score reflect forgiveness recently forgot small charge card rarely use linked old email bills reported tanked credit score called able fees removed sure forgiveness carry credit score',
 'need help budgetting getting debt long story short years old working full time job terrible money management issues course good amount debt feel backpedal help advice please rude comments know debt isnt lot reaching hopes reigning gets worse monthly income monthly bills rent utilities car payment car insurance phone internet gas fuel food groceries medical debt payments established payments established pay remaining this debt variety different th',
 'year old male almost two year fiance recently dropped pursue school working full ti

Unfortuntely a few get cut-off mid-word/mid-setence so limiting by characters may not be the best option. How about if we try to limit the text length by word count instead?

In [12]:
# Compute the mean and median word counts
txt_wrd_mean = df['clean_text'].apply(lambda x: x.split()).map(len).mean()
txt_wrd_median = df['clean_text'].apply(lambda x: x.split()).map(len).median()

print('Mean Word Length: {}\n\nMedian Word length: {}'.format(txt_wrd_mean, txt_wrd_median))

Mean Word Length: 64.79432275807878

Median Word length: 47.0


In [13]:
# Observe the top 10 texts with most number of word
txt_wrd_list= {}
for n, x in enumerate(df['clean_text']):
    txt_wrd_list[n] = len(x.split())
sorted(txt_wrd_list.items(), key = lambda x: x[1], reverse = True)[0:10]

[(858, 1338),
 (1191, 944),
 (2435, 939),
 (6524, 918),
 (2608, 819),
 (9512, 758),
 (9846, 744),
 (4495, 734),
 (6921, 732),
 (9090, 688)]

In [14]:
# Count the number of texts with fewer than 150 words
word_count_ = len(df[df['clean_text'].apply(lambda x: x.split()).map(len)<150])
total_len = len(df)

print('Percentage of texts with word count less than 150: {0:.2f}%'.format((word_count_/total_len)*100))

Percentage of texts with word count less than 150: 90.11%


In [15]:
for text in df['clean_text'][0:5]:
    print(' '.join(text.split()[0:150]))
    print('')

ways make extra side money

year update legally blind going homeless one job making month please help

kicked found last night home ive staying going sale days currently working due work related injury returning make week mostly appartments around high mid range currently working car sure subreddit tips looking places live bugdet money properly anything else thanks edit grammer errors

online savings account hello looking recommendations online savings account basically something easily transfer money limits taking money working changing spending habits would like commit certain percentage income every pay period still working getting debts paid first total thankfully adjusting better habits like bringing lunch instead ordering thank

tools managing incomes expenses



In [ ]:
df['cleaned_text'] = [' '.join(text.split()[0:150]) for text in df['clean_text']]

Finally, let's save the data as a .csv file for later use.

In [ ]:
# Save as .csv file
df.to_csv(r'C:\Users\joshua\Downloads\Data\reddit\reddit_pf3.csv')

In [ ]:
df = pd.read_csv(r'C:\Users\joshua\Downloads\Data\reddit\reddit_pf3.csv', engine='python', index_col=[0], parse_dates = True)

- **Tokenization**

Tokenization is essentially the process of segmenting a text into pieces, such as words, phrases, symbols, etc. 

Let's create a list of the tokens for each submission title.

In [6]:
df.head()

date      id                                          self_text  \
0  2018-09-19  9h6whn                                                      
1  2018-09-19  9h29g7                                                      
2  2018-09-19  9h6vyv  So i just found out last night the home ive be...   
3  2018-09-19  9h6vs4  Hello! Looking for recommendations for an onli...   
4  2018-09-19  9h6v48                                                      

                                                text      time  \
0                    Ways to make extra side money?   12:57 PM   
1  (Year UPDATE) Legally blind, going homeless, h...  12:56 AM   
2  19, being kicked out So i just found out last ...  12:55 PM   
3  Online Savings Account? Hello! Looking for rec...  12:54 PM   
4           Tools for Managing Incomes and Expenses   12:52 PM   

                                               title    topic  upvotes  \
0                     Ways to make extra side money?  unknown        1   
1  (Year UPDATE) Legally blind, going homeless, h...    Other       16   
2                               19, being kicked out    Other        2   
3                            Online Savings Account?   Saving        1   
4            Tools for Managing Incomes and Expenses    Other        0   

                                          clean_text  
0                         ways make extra side money  
1  year update legally blind going homeless one j...  
2  kicked found last night home ive staying going...  
3  online savings account hello looking recommend...  
4                    tools managing incomes expenses

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def create_tokens(text):
    doc = nlp(text)
    tokens = [token for token in doc]
    return tokens

df['tokenized_text'] = df['clean_text'].apply(lambda x: create_tokens(x))

- **Lemmatization**

Lemmas are the "base form" of a word. 

Ex. walk, walked, walking, walks would all be derived from the base form 'walk'. 

Using the tokens that we generated in the column 'tokenized_title', let's next lemmatize the tokens.

In [ ]:
def lemmatize(text):
    # Make sure to remove pronouns (ex. he, she) before returning the lemmas
    lemmas = [token.lemma_ for token in text if token.lemma_ not in '-PRON-']
    return lemmas

df['lemmatized_text'] = df['tokenized_text'].apply(lambda x: lemmatize(x))

- **Named Entity Recognition**

Named entities are real-world objects that have a name, such as a person, country, or company. spaCy is able to recognize different types of named entities in a document and can return features such as the label (ex. ORG - organization, GPE - geopolitical entity).

In [ ]:
def create_NER(text, label = False):
    doc = nlp(text)
    if label is False:
        NER_list = [(ent.text) for ent in doc.ents]
    else:
        NER_list = [(ent.label_) for ent in doc.ents]
    return NER_list    

df['named_entities'] = df['clean_text'].apply(lambda x: create_NER(x))
df['entity_labels'] = df['clean_text'].apply(lambda x: create_NER(x, label = True))

In [ ]:
#df.drop('tokenized_text', axis=1).to_pickle(r'C:\Users\Joshua\Pickle_files\df')